[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab7/lab7.ipynb)
# Práctica 7: Residual neural networks


### Pre-requisitos. Instalar paquetes

Para la primera parte de este Laboratorio 7 necesitaremos TensorFlow y TensorFlow-Datasets. Además, como habitualmente, fijaremos la semilla aleatoria para asegurar la reproducibilidad de los experimentos.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Fijamos la semilla para poder reproducir los resultados
import os
import numpy as np
import random
seed=1234
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

Además, cargamos también APIs que vamos a emplear para que el código quede más legible

In [ ]:
#API de Keras, modelo Sequential y la capa Dense 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense 
#Para mostrar gráficas
from matplotlib import pyplot

### Carga del conjunto de datos

De nuevo, seguimos empleando el conjunto *german_credit_numeric* ya empleado en los laboratorios anteriores.


In [ ]:
# TODO: Carga el conjunto german_credit como ds_train
# Indica además un tamaño de batch de 128 y que se repita indefinidamente
ds_train = ...

## Visualización del desvanecimiento del gradiente
Las Residual neural networks se ocupan de que los gradientes no se desvanezcan cuando la red es muy profunda. Vamos a visualizar este problema creando una red profunda y mostrando las dimensiones de los gradientes que llegan a cada capa.

Para ello debemos registrar las dimensiones de los gradientes a lo largo de entrenamiento. Crearemos un nuevo tipo de modelo, que va registrando los gradientes a medida que es entrenado. Nuestra nueva clase heredará de `tf.keras.models.Sequential`.

In [ ]:
class GradientLoggingSequentialModel(tf.keras.models.Sequential):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # En la inicialización instanciamos una nueva variable en la que
        # registraremos el historial de tamaños de gradientes de cada capa
        self.gradient_history = {}
    
    def compile(self, **kwargs):
        result = super().compile(**kwargs)
        # Una vez sabemos la arquitectura, podemos inicializar la historia
        # de gradientes de cada capa a una lista vacía.
        for l in self.layers:
            self.gradient_history[l.name] = []
        return result
        
    def _save_gradients(self, gradients):
        # A cada paso de entrenamiento llamaremos a esta función para que
        # registre los gradientes.
        # En la lista gradients se encuentran los gradientes de las distintas
        # capas por orden. Cada capa l tendrá un número de gradientes que
        # concidirá con l.trainable_variables.
        # Teniendo esto en cuenta, recorremos los gradientes, calculamos su
        # tamaño y guardamos la media de tamaños de cada capa en el histórico
        i = 0
        for layer in self.layers:
            gradient_sizes = []
            for lw in layer.trainable_variables:
                g_size = np.linalg.norm(gradients[i].numpy())
                gradient_sizes.append(g_size)
                i += 1
            mean_gradient_size = np.mean(gradient_sizes)
            self.gradient_history[layer.name].append(mean_gradient_size)
        
    def train_step(self, data):
        # Haremos un paso de entrenamiento personalizado basado en 
        # https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit#a_first_simple_example
        # Dejaremos el ejemplo tal cual, añadiendo tan solo la llamada a
        # _save_gradients una vez que disponemos de los gradientes
        
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        
        # Llamada añadida para grabar los gradientes.
        self._save_gradients(gradients)
        
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

### Creamos un modelo *GradientLoggingSequentialModel*
Creamos un modelo *Sequential* para ajustar a los datos de entrada siguiendo las especificaciones dadas.

In [ ]:
# TODO - Define en model una red GradientLoggingSequentialModel con 20 capas ocultas, con activación sigmoide, con 10 unidades por capa.

model = ...

#Construimos el modelo y mostramos 
model.build()
print(model.summary())

### Entrenamiento del modelo
Vamos a establecer la función de pérdida (entropía cruzada binaria), el optimizador (SGD con LR $10^{-3}$) y la métrica que nos servirá para evaluar el rendimiento del modelo entrenado (área bajo la curva).

In [ ]:
#TODO - Compila el modelo. Utiliza la opción run_eagerly=True para que se puedan registrar los gradientes a cada paso
model.compile(...)

Entrenamos el modelo usando model.fit

In [ ]:
#TODO - entrenar el modelo utilizando 8 steps por epoch. Con 10 epochs nos valdrá para comprobar el desvanecimiento de gradientes.
model.fit(...)

Ahora que hemos hecho algunos pasos de entrenamiento, representamos el tamaño medio de los pesos de cada capa.

In [ ]:
# Ahora accedemos al historial de gradientes y representamos el tamaño medio de los gradientes de cada capa.
pyplot.figure(figsize=(14, 6), dpi=80)
pyplot.boxplot(model.gradient_history.values())
pyplot.yscale('log')
pyplot.xticks(ticks=range(1,len(model.gradient_history)+1), labels=model.gradient_history.keys())
pyplot.show()

## Comparativa
 - ¿Qué observas en los pesos?
 - ¿Ocurre lo mismo utilizando ReLU como función de activación de las capas ocultas?
 - Repite la prueba con las distintas [funciones de activación](https://keras.io/api/layers/activations/) que tengan sentido.
 - (OPCIONAL) Alarga el entrenamiento y prueba distintos optimizadores para intentar que el modelo entrene correctamente.